In [1]:
!python libraries.py

2024-08-28 13:33:41,033 - INFO - Upgrading pip...
2024-08-28 13:33:41,033 - INFO - Running pip command: install --upgrade pip
2024-08-28 13:33:42,250 - INFO - Command 'install --upgrade pip' executed successfully.
2024-08-28 13:33:42,506 - INFO - pip upgraded successfully.
2024-08-28 13:33:42,506 - INFO - Installing library numpy...
2024-08-28 13:33:42,506 - INFO - Running pip command: install numpy
2024-08-28 13:33:43,574 - INFO - Command 'install numpy' executed successfully.
2024-08-28 13:33:43,574 - INFO - numpy installed successfully.
2024-08-28 13:33:43,574 - INFO - Installing library pandas...
2024-08-28 13:33:43,574 - INFO - Running pip command: install pandas
2024-08-28 13:33:44,666 - INFO - Command 'install pandas' executed successfully.
2024-08-28 13:33:44,666 - INFO - pandas installed successfully.
2024-08-28 13:33:44,666 - INFO - Installing library matplotlib...
2024-08-28 13:33:44,666 - INFO - Running pip command: install matplotlib
2024-08-28 13:33:45,740 - INFO - Comman

In [2]:
import tensorflow as tf
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16903437380112853314
xla_global_id: -1
]


In [4]:
import os
import pandas as pd
import numpy as np
import utils

In [5]:
data_dir = './data'
measurement_data_dir = f'{data_dir}/measurement'
os.listdir(measurement_data_dir)

trips_dir = os.path.join(measurement_data_dir, 'trips')
os.listdir(trips_dir)

csvs_list = [f for f in os.listdir(trips_dir) if f.endswith('.csv')]
trips = utils.csvs2dfs(
    trips_dir=trips_dir, 
    csvs_list=csvs_list
)

for i in range(len(trips)):
    trips[i].columns = [col.replace('_', '') for col in trips[i].columns]

bounds_SOC = utils.data_cleaning(
    dfs=trips,
    steady_cols_to_drop=['minSoC', 'maxSoC']
)

min_SOC = bounds_SOC[0]
max_SOC = bounds_SOC[1]

In [6]:
y_train = []
for i in range(len(trips)):
    yt = trips[i]['SoC'].values
    # drop della colonna poiche' ho gia' salvato i valori in y_train cosi' da rendere piu' semplice la costruzione di X_train
    trips[i].drop(['SoC'], axis=1, inplace=True)
    y_train.append(yt)

y_train_shapes = []
for i in range(len(y_train)):
    y_train_shapes.append(y_train[i].shape)
y_train_shapes

[(10090,),
 (14130,),
 (6706,),
 (4124,),
 (13667,),
 (31645,),
 (20933,),
 (28059,),
 (18344,),
 (14176,),
 (14245,),
 (16384,),
 (7160,),
 (6946,),
 (22348,),
 (19066,),
 (6687,),
 (8808,),
 (15876,),
 (17196,),
 (19829,),
 (18388,),
 (10426,),
 (5322,),
 (7627,),
 (16122,),
 (20005,),
 (17436,),
 (13390,),
 (13974,),
 (11870,),
 (16722,),
 (32518,),
 (16113,),
 (15794,),
 (29550,),
 (10195,),
 (13521,),
 (22899,),
 (29140,),
 (56102,),
 (20233,),
 (7534,),
 (32256,),
 (3545,),
 (38220,),
 (18223,),
 (15286,),
 (15610,),
 (11095,),
 (11911,),
 (14029,),
 (10397,),
 (11993,),
 (11133,),
 (9780,),
 (10219,),
 (8050,),
 (14690,),
 (13665,),
 (9686,),
 (9209,),
 (10969,),
 (7958,),
 (5480,),
 (7338,),
 (13626,),
 (28523,),
 (14173,),
 (16429,)]

In [7]:
len_y_train_flatten = 0
for i in range(len(y_train_shapes)):
    len_y_train_flatten += y_train_shapes[i][0]
len_y_train_flatten

1094793

In [8]:
y_train_flatten = np.zeros(len_y_train_flatten)
k = 0
for i in range(len(y_train)):
    yt = y_train[i]
    for j in range(len(yt)):
        y_train_flatten[k] = yt[j]
        k = k+1
y_train_flatten

array([86.9, 86.9, 86.9, ..., 48.8, 48.8, 48.8])

In [9]:
X_train = []
for i in range(len(trips)):
    xt = trips[i].values
    X_train.append(xt)

X_train_shapes = []
for i in range(len(X_train)):
    X_train_shapes.append(X_train[i].shape)
X_train_shapes

[(10090, 19),
 (14130, 19),
 (6706, 19),
 (4124, 19),
 (13667, 19),
 (31645, 19),
 (20933, 19),
 (28059, 19),
 (18344, 19),
 (14176, 19),
 (14245, 19),
 (16384, 19),
 (7160, 19),
 (6946, 19),
 (22348, 19),
 (19066, 19),
 (6687, 19),
 (8808, 19),
 (15876, 19),
 (17196, 19),
 (19829, 19),
 (18388, 19),
 (10426, 19),
 (5322, 19),
 (7627, 19),
 (16122, 19),
 (20005, 19),
 (17436, 19),
 (13390, 19),
 (13974, 19),
 (11870, 19),
 (16722, 19),
 (32518, 19),
 (16113, 19),
 (15794, 19),
 (29550, 19),
 (10195, 19),
 (13521, 19),
 (22899, 19),
 (29140, 19),
 (56102, 19),
 (20233, 19),
 (7534, 19),
 (32256, 19),
 (3545, 19),
 (38220, 19),
 (18223, 19),
 (15286, 19),
 (15610, 19),
 (11095, 19),
 (11911, 19),
 (14029, 19),
 (10397, 19),
 (11993, 19),
 (11133, 19),
 (9780, 19),
 (10219, 19),
 (8050, 19),
 (14690, 19),
 (13665, 19),
 (9686, 19),
 (9209, 19),
 (10969, 19),
 (7958, 19),
 (5480, 19),
 (7338, 19),
 (13626, 19),
 (28523, 19),
 (14173, 19),
 (16429, 19)]

In [10]:
len_X_train_flatten = 0
for i in range(len(X_train_shapes)):
    len_X_train_flatten += X_train_shapes[i][0]
len_X_train_flatten

1094793

In [11]:
X_train_shapes[0][1]

19

In [13]:
X_train_flatten = np.zeros((len_X_train_flatten, X_train_shapes[0][1]))
k = 0
for i in range(len(X_train)):
    xt = X_train[i].flatten()
    for j in range(len(xt)):
        X_train_flatten[k] = xt[j]
        k = k+1
X_train_flatten

IndexError: index 19 is out of bounds for axis 0 with size 19

In [ ]:
import sklearn